In [28]:
import boto3
import codecs
import csv
import datetime
import holidays
import json

### To adjust the filter for dates in the USFR spider

In [6]:
def next_business_day(date, time_span, time_unit, country_code):
    ONE_DAY = datetime.timedelta(days=1)
    HOLIDAYS = holidays.CountryHoliday(country_code)
    next_day = date + ONE_DAY
    while next_day.weekday() in holidays.WEEKEND or next_day in HOLIDAYS:
       next_day += ONE_DAY
    return next_day

def create_date_list(start_date, to_date, time_span, time_unit, country_code):
    dates = []
    start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()
    to_date = datetime.datetime.strptime(to_date, '%Y-%m-%d').date()
    while start_date < to_date:
        start_date = next_business_day(start_date, time_span, time_unit, country_code)
        dates.append(start_date)
    return dates

In [21]:
path = "/home/propietari/Documents/GitHub/policy-data-analyzer/tasks/Scrapy/scrapy_official_newspapers/output/"
file_name = "USFR_20210310_1.csv"
file = path + file_name

dates_dict = {}

with open(file, 'r', errors="surrogateescape") as f:   
    reader = csv.reader(f)
    for row in reader: #unicode(reader, errors='ignore'):
        dates_dict[datetime.datetime.strptime(row[6], '%Y-%m-%d').date()] = 0
        
from_date = "2010-01-01"
to_date = "2010-01-07"
country_code = "US"
for day in create_date_list(from_date, to_date, 1, "days", country_code):
    if day in dates_dict:
        print(day)

2010-01-04
2010-01-05


In [9]:
type(from_date)

str

### To remove duplicates from csv by file name

In [13]:
path = "/home/propietari/Documents/GitHub/policy-data-analyzer/tasks/Scrapy/scrapy_official_newspapers/output/"
file_name = "USFR_20210310.csv"
file = path + file_name

no_dups_dict = {}

i = 0

with open(file, 'r') as f:  #, errors="surrogateescape" 
    reader = csv.reader(f)
    for row in reader: #unicode(reader, errors='ignore'):
        i += 1
        no_dups_dict[row[3]] = row
        
print("* Number of entries:", i, "\n* Number of unique entries:", len(no_dups_dict))

file_name = "USFR_metadata.csv"
file = path + file_name
i = 0
with open(file, 'w') as f:
    writer = csv.writer(f)
    for key, value in no_dups_dict.items():
#         value[0] = value[0].replace("\udca0", "")
        try:
            writer.writerow([ value[0], value[1], value[2], value[3], value[4], value[5], value[6], value[7], value[8], value[9], value[10], value[11]])
        except:
            print("* ", value[3], "could not be saved. ", value[0])

* Number of entries: 18202 
* Number of unique entries: 15594


### To upload csv form S3

In [18]:
# Connection to AWS
folder = '/home/propietari/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_WRI.json' # TODO: Change to your filename
file = folder + file_name

with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
s3BucketName = "wri-nlp-policy"
region = 'us-east-1'

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [39]:
import boto3
import codecs
import csv

# Connection to AWS
folder = '/home/propietari/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_WRI.json' # TODO: Change to your filename
file = folder + file_name

with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
s3BucketName = "wri-nlp-policy"
region = 'us-east-1'

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

# Load file

# You can adapt it to get the whole contents of each row of the bucket updating a list or lists that replicates
# the csv 

# The counter just for testing purposes
i = 0

file_key = "India_metadata.csv"

obj = s3.Object(bucket_name = s3BucketName, key = file_key)
for row in csv.reader(codecs.getreader("utf-8")(obj.get()['Body'])):
    if i < 5:
        print(row[3])
    i += 1


859644d69fdd0b1e66f1dcf5a42f2d9a3f31fe90.pdf
876b66a43ea68e055193465940377107b9cd4fcc.pdf
e898428130c73dd8be1dfc58fe2198ab388ebd96.pdf
fe8b9097194caaed0b8985f4a4dddeca06833fa4.pdf
afca018efb184ada2624bba04c111a054a068037.pdf
